In [0]:
#connecting via Service Principle (using secret scope)
spark.conf.set("fs.azure.account.auth.type.adlsg2datalake.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.adlsg2datalake.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.adlsg2datalake.dfs.core.windows.net", "37a372de-1804-4a80-b98f-54ba7ef17ef3")
spark.conf.set("fs.azure.account.oauth2.client.secret.adlsg2datalake.dfs.core.windows.net", dbutils.secrets.get(scope="databricks_secret_scope_soham", key="secret-adlsg2-service-principle-value"))
spark.conf.set("fs.azure.account.oauth2.client.endpoint.adlsg2datalake.dfs.core.windows.net", "https://login.microsoftonline.com/9adebd2f-da4d-4f91-9e68-91d049a77ef2/oauth2/token")

In [0]:
#display(dbutils.fs.ls("abfss://databricks-hello-world@adlsg2datalake.dfs.core.windows.net"))
display(dbutils.fs.ls("abfss://databricks-hello-world@adlsg2datalake.dfs.core.windows.net/source_json_sample_data/"))
#display(dbutils.fs.ls("abfss://databricks-hello-world@adlsg2datalake.dfs.core.windows.net/source_json_cricket_files/"))


In [0]:
df = spark.read.option("multiline", "true").format("json").load("abfss://databricks-hello-world@adlsg2datalake.dfs.core.windows.net/source_json_cricket_files/")
 

In [0]:
#df.display()
#df.count()
df.createOrReplaceTempView("RAW_JSON")

In [0]:
%sql
SELECT
  INN_IDX AS INN_NO,
  --INN_VAL,
  INN_VAL.TEAM,
  --INN_VAL.POWERPLAYS,
  OVER_IDX,
  --OVER_VAL,
  --OVER_VAL.*
  OVER_VAL.OVER AS OVER_NO,
  (DEL_IDX + 1) AS DELIVERY_NO, 
  DEL_VAL.BATTER,
  DEL_VAL.BOWLER,
  DEL_VAL.EXTRAS,
  DEL_VAL.NON_STRIKER,
  --DEL_VAL.RUNS,
  DEL_VAL.RUNS.BATTER AS RUN_BATTED,
  DEL_VAL.RUNS.EXTRAS AS RUN_EXTRAS,
  DEL_VAL.RUNS.TOTAL AS RUN_TOTAL,
  DEL_VAL.WICKETS[0].PLAYER_OUT,
  DEL_VAL.WICKETS[0].KIND,
  --DEL_VAL.WICKETS[0].FIELDERS,
  DEL_VAL.WICKETS[0].FIELDERS[0].NAME AS FIELDER_1,
  DEL_VAL.WICKETS[0].FIELDERS[0].SUBSTITUTE AS SUBSTITUTE,
  DEL_VAL.WICKETS[0].FIELDERS[1].NAME AS FIELDER_2
  --DEL_VAL.WICKETS
  --DEL_VAL.*
FROM
  RAW_JSON
LATERAL VIEW POSEXPLODE(INNINGS) AS INN_IDX, INN_VAL
LATERAL VIEW POSEXPLODE(INN_VAL.OVERS) AS OVER_IDX, OVER_VAL 
LATERAL VIEW POSEXPLODE(OVER_VAL.DELIVERIES) AS DEL_IDX, DEL_VAL
;

In [0]:
# Method 1:
#Saving the files using _sqldf in ADLSg2 in Delta format

_sqldf \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .save("abfss://databricks-hello-world@adlsg2datalake.dfs.core.windows.net/target_conformed_delta_lakehouse/_sqldf__write/Innings_Details") ;

In [0]:
#spark.read.format("delta").load("abfss://databricks-hello-world@adlsg2datalake.dfs.core.windows.net/target_conformed_delta_lakehouse/_sqldf__write/Innings_Details").display();

df = spark.read.format("delta").load("abfss://databricks-hello-world@adlsg2datalake.dfs.core.windows.net/target_conformed_delta_lakehouse/_sqldf__write/Innings_Details");

In [0]:
df.count();


In [0]:
%sql
--Method 2:
--Creating Managed Table in ADLSg2 in Delta format
CREATE DATABASE IF NOT EXISTS CRICKET_DB;
USE CRICKET_DB;


In [0]:
%sql
CREATE TABLE IF NOT EXISTS CRICKET_DB.CRICKET_DELIVERIES
AS
SELECT
  INN_IDX AS INN_NO,
  --INN_VAL,
  INN_VAL.TEAM,
  --INN_VAL.POWERPLAYS,
  OVER_IDX,
  --OVER_VAL,
  --OVER_VAL.*
  OVER_VAL.OVER AS OVER_NO,
  (DEL_IDX + 1) AS DELIVERY_NO, 
  DEL_VAL.BATTER,
  DEL_VAL.BOWLER,
  DEL_VAL.EXTRAS,
  DEL_VAL.NON_STRIKER,
  --DEL_VAL.RUNS,
  DEL_VAL.RUNS.BATTER AS RUN_BATTED,
  DEL_VAL.RUNS.EXTRAS AS RUN_EXTRAS,
  DEL_VAL.RUNS.TOTAL AS RUN_TOTAL,
  DEL_VAL.WICKETS[0].PLAYER_OUT,
  DEL_VAL.WICKETS[0].KIND,
  --DEL_VAL.WICKETS[0].FIELDERS,
  DEL_VAL.WICKETS[0].FIELDERS[0].NAME AS FIELDER_1,
  DEL_VAL.WICKETS[0].FIELDERS[0].SUBSTITUTE AS SUBSTITUTE,
  DEL_VAL.WICKETS[0].FIELDERS[1].NAME AS FIELDER_2
  --DEL_VAL.WICKETS
  --DEL_VAL.*
FROM
  RAW_JSON
LATERAL VIEW POSEXPLODE(INNINGS) AS INN_IDX, INN_VAL
LATERAL VIEW POSEXPLODE(INN_VAL.OVERS) AS OVER_IDX, OVER_VAL 
LATERAL VIEW POSEXPLODE(OVER_VAL.DELIVERIES) AS DEL_IDX, DEL_VAL
;

In [0]:
%sql
--SELECT * FROM CRICKET_DB.CRICKET_DELIVERIES LIMIT 100;
--SELECT * FROM CRICKET_DB.CRICKET_DELIVERIES WHERE FIELDER_1 IS NOT NULL;
SELECT COUNT(1) FROM CRICKET_DB.CRICKET_DELIVERIES ;


In [0]:
%sql
--Method 2:
--Creating Managed Table in ADLSg2 in Delta format

--try in premium databricks. not working in trial version

CREATE TABLE CRICKET_DB.CRICKET_DELIVERIES_EXTERNAL
LOCATION 'abfss://databricks-hello-world@adlsg2datalake.dfs.core.windows.net/target_conformed_delta_lakehouse/cricket_db_external_tables/Innings_Details'
AS
SELECT
  INN_IDX AS INN_NO,
  --INN_VAL,
  INN_VAL.TEAM,
  --INN_VAL.POWERPLAYS,
  OVER_IDX,
  --OVER_VAL,
  --OVER_VAL.*
  OVER_VAL.OVER AS OVER_NO,
  (DEL_IDX + 1) AS DELIVERY_NO, 
  DEL_VAL.BATTER,
  DEL_VAL.BOWLER,
  DEL_VAL.EXTRAS,
  DEL_VAL.NON_STRIKER,
  --DEL_VAL.RUNS,
  DEL_VAL.RUNS.BATTER AS RUN_BATTED,
  DEL_VAL.RUNS.EXTRAS AS RUN_EXTRAS,
  DEL_VAL.RUNS.TOTAL AS RUN_TOTAL,
  DEL_VAL.WICKETS[0].PLAYER_OUT AS PLAYER_OUT,
  DEL_VAL.WICKETS[0].KIND AS KIND,
  --DEL_VAL.WICKETS[0].FIELDERS,
  DEL_VAL.WICKETS[0].FIELDERS[0].NAME AS FIELDER_1,
  DEL_VAL.WICKETS[0].FIELDERS[0].SUBSTITUTE AS SUBSTITUTE,
  DEL_VAL.WICKETS[0].FIELDERS[1].NAME AS FIELDER_2
  --DEL_VAL.WICKETS
  --DEL_VAL.*
FROM
  RAW_JSON
LATERAL VIEW POSEXPLODE(INNINGS) AS INN_IDX, INN_VAL
LATERAL VIEW POSEXPLODE(INN_VAL.OVERS) AS OVER_IDX, OVER_VAL 
LATERAL VIEW POSEXPLODE(OVER_VAL.DELIVERIES) AS DEL_IDX, DEL_VAL
;